In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
table_scan_data = pd.read_csv("../data/train/TableScan.csv")
table_data = pd.read_csv("../data/train/table_meta.csv")
columns_data = pd.read_csv("../data/train/column_meta.csv")
chunk_meta = pd.read_csv("../data/train/segment_meta.csv")

joined_data = table_scan_data.merge(table_data, on=["TABLE_NAME"], how="left")
joined_data = joined_data.merge(columns_data, on=["TABLE_NAME", "COLUMN_NAME"], how="left")

# only look at the encoding of the first chunk
chunkrows = chunk_meta.loc[(chunk_meta['CHUNK_ID'] == 0)]
joined_data = joined_data.merge(chunkrows, on=["TABLE_NAME", "COLUMN_NAME"], how="left")

joined_data = joined_data.rename(columns={"INPUT_ROWS_LEFT": "INPUT_ROWS", "CHUNK_SIZE": "MAX_CHUNK_SIZE",
                                          "COLUMN_DATA_TYPE": "DATA_TYPE", "ENCODING_TYPE": "ENCODING"})

# remove the rows with ExpressionEvaluator queries from the test data since we don't have any in the test data
joined_data = joined_data.loc[(joined_data['SCAN_IMPLEMENTATION'] != 'ExpressionEvaluator')]

# explicitly add selectivity
joined_data['SELECTIVITY'] = (joined_data['OUTPUT_ROWS'] / joined_data['INPUT_ROWS'])
joined_data['SELECTIVITY'].fillna(0, inplace=True)

In [ ]:
joined_data.describe()

In [ ]:
ohe_data = joined_data.drop(labels=['TABLE_NAME', 'COLUMN_NAME'], axis=1)
ohe_data = pd.get_dummies(ohe_data, columns=['SCAN_TYPE', 'DATA_TYPE', 'ENCODING', 'SCAN_IMPLEMENTATION', 'COMPRESSION_TYPE'])
ohe_data.head()

In [ ]:
ohe_data.describe()

In [ ]:
plt.scatter(ohe_data['INPUT_ROWS'],ohe_data['RUNTIME_NS'])

In [ ]:
plt.scatter(ohe_data['SELECTIVITY'],ohe_data['RUNTIME_NS'])
#plt.ylim(0, 100000)

In [ ]:
weird_data = ohe_data.loc[(ohe_data['RUNTIME_NS'] < 100000) & (ohe_data['SELECTIVITY'] > 0) ]
weird_data.head()
plt.scatter(weird_data['SELECTIVITY'],weird_data['RUNTIME_NS'])

In [ ]:
# looking at impact of the datatype

floats = ohe_data.loc[ohe_data['DATA_TYPE_float'] == 1]
ints = ohe_data.loc[ohe_data['DATA_TYPE_int'] == 1]
strings = ohe_data.loc[ohe_data['DATA_TYPE_string'] == 1]
longs = ohe_data.loc[ohe_data['DATA_TYPE_long'] == 1]
doubles = ohe_data.loc[ohe_data['DATA_TYPE_double'] == 1]

fig, axs = plt.subplots(2, 3, sharex=False, sharey=True, figsize=(15,9))
fig.suptitle('Datatype')
axs[0,0].scatter(floats['INPUT_ROWS'], floats['RUNTIME_NS'], color='r', alpha=0.5)
axs[0, 0].set_title('Float')
axs[0,1].scatter(ints['INPUT_ROWS'], ints['RUNTIME_NS'], color='b', alpha=0.5)
axs[0, 1].set_title('Int')
axs[0,2].scatter(strings['INPUT_ROWS'], strings['RUNTIME_NS'], color='g', alpha=0.5)
axs[0, 2].set_title('String')
axs[1,0].scatter(doubles['INPUT_ROWS'], doubles['RUNTIME_NS'], color='orange',alpha=0.5)
axs[1,0].set_title('Double')
axs[1,1].scatter(longs['INPUT_ROWS'], longs['RUNTIME_NS'], color='purple', alpha=0.5)
axs[1,1].set_title('Long')

# for labels
fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axis
plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
plt.xlabel("#Input Rows")
plt.ylabel("Time [ns]")
plt.show()

In [ ]:
# looking at impact of the datatype

floats = ohe_data.loc[ohe_data['DATA_TYPE_float'] == 1]
ints = ohe_data.loc[ohe_data['DATA_TYPE_int'] == 1]
strings = ohe_data.loc[ohe_data['DATA_TYPE_string'] == 1]
longs = ohe_data.loc[ohe_data['DATA_TYPE_long'] == 1]
doubles = ohe_data.loc[ohe_data['DATA_TYPE_double'] == 1]

fig, axs = plt.subplots(2, 3, sharex=True, sharey=True, figsize=(15,9))
fig.suptitle('Datatype')
axs[0,0].scatter(floats['SELECTIVITY'], floats['RUNTIME_NS'], color='r', alpha=0.5)
axs[0, 0].set_title('Float')
axs[0,1].scatter(ints['SELECTIVITY'], ints['RUNTIME_NS'], color='b', alpha=0.5)
axs[0, 1].set_title('Int')
axs[0,2].scatter(strings['SELECTIVITY'], strings['RUNTIME_NS'], color='g', alpha=0.5)
axs[0, 2].set_title('String')
axs[1,0].scatter(doubles['SELECTIVITY'], doubles['RUNTIME_NS'], color='orange',alpha=0.5)
axs[1,0].set_title('Double')
axs[1,1].scatter(longs['SELECTIVITY'], longs['RUNTIME_NS'], color='purple', alpha=0.5)
axs[1,1].set_title('Long')
#axs[0,0].set_ylim([0, 100000])
# for labels
fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axis
plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
plt.xlabel("Selectivity")
plt.ylabel("Time")
plt.show()

In [ ]:
# looking at impact of the datatype

dicts = ohe_data.loc[ohe_data['ENCODING_Dictionary'] == 1]
fsdicts = ohe_data.loc[ohe_data['ENCODING_FixedStringDictionary'] == 1]
forefs = ohe_data.loc[ohe_data['ENCODING_FrameOfReference'] == 1]
lz4s = ohe_data.loc[ohe_data['ENCODING_LZ4'] == 1]
runls = ohe_data.loc[ohe_data['ENCODING_RunLength'] == 1]
noencs = ohe_data.loc[ohe_data['ENCODING_Unencoded'] == 1]

fig, axs = plt.subplots(2, 3, sharex=True, sharey=True, figsize=(15,9))
#ax=fig.add_axes([0,0,1,1])
fig.suptitle('Encoding')
axs[0,0].scatter(dicts['INPUT_ROWS'], dicts['RUNTIME_NS'], color='r', alpha=0.5)
axs[0, 0].set_title('Dictionary')
axs[0,1].scatter(fsdicts['INPUT_ROWS'], fsdicts['RUNTIME_NS'], color='b', alpha=0.5)
axs[0, 1].set_title('FixedStringDict')
axs[0,2].scatter(forefs['INPUT_ROWS'], forefs['RUNTIME_NS'], color='g', alpha=0.5)
axs[0, 2].set_title('FrameOfReference')
axs[1,0].scatter(lz4s['INPUT_ROWS'], lz4s['RUNTIME_NS'], color='black',alpha=0.5)
axs[1, 0].set_title('LZ4')
axs[1,1].scatter(runls['INPUT_ROWS'], runls['RUNTIME_NS'], color='purple', alpha=0.5)
axs[1, 1].set_title('RunLength')
axs[1,2].scatter(noencs['INPUT_ROWS'], noencs['RUNTIME_NS'], color='orange',alpha=0.5)
axs[1, 2].set_title('Unencoded')

# for labels
fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axis
plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
plt.xlabel("#Input Rows")
plt.ylabel("Time")
    
plt.show()


In [ ]:
# looking at impact of the datatype

dicts = ohe_data.loc[ohe_data['ENCODING_Dictionary'] == 1]
fsdicts = ohe_data.loc[ohe_data['ENCODING_FixedStringDictionary'] == 1]
forefs = ohe_data.loc[ohe_data['ENCODING_FrameOfReference'] == 1]
lz4s = ohe_data.loc[ohe_data['ENCODING_LZ4'] == 1]
runls = ohe_data.loc[ohe_data['ENCODING_RunLength'] == 1]
noencs = ohe_data.loc[ohe_data['ENCODING_Unencoded'] == 1]

fig, axs = plt.subplots(2, 3, sharex=True, sharey=True, figsize=(15,9))
#ax=fig.add_axes([0,0,1,1])
fig.suptitle('Encoding')
axs[0,0].scatter(dicts['SELECTIVITY'], dicts['RUNTIME_NS'], color='r', alpha=0.5)
axs[0, 0].set_title('Dictionary')
axs[0,1].scatter(fsdicts['SELECTIVITY'], fsdicts['RUNTIME_NS'], color='b', alpha=0.5)
axs[0, 1].set_title('FixedStringDict')
axs[0,2].scatter(forefs['SELECTIVITY'], forefs['RUNTIME_NS'], color='g', alpha=0.5)
axs[0, 2].set_title('FrameOfReference')
axs[1,0].scatter(lz4s['SELECTIVITY'], lz4s['RUNTIME_NS'], color='black',alpha=0.5)
axs[1, 0].set_title('LZ4')
axs[1,1].scatter(runls['SELECTIVITY'], runls['RUNTIME_NS'], color='purple', alpha=0.5)
axs[1, 1].set_title('RunLength')
axs[1,2].scatter(noencs['SELECTIVITY'], noencs['RUNTIME_NS'], color='orange',alpha=0.5)
axs[1, 2].set_title('Unencoded')
#axs[0,0].set_ylim([0, 100000])

# for labels
fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axis
plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
plt.xlabel("Selectivity")
plt.ylabel("Time")
    
plt.show()


In [ ]:
# looking at impact of the scan type
#SCAN_IMPLEMENTATION_ColumnBetween	SCAN_IMPLEMENTATION_ColumnLike	SCAN_IMPLEMENTATION_ColumnVsColumn	SCAN_IMPLEMENTATION_ColumnVsValue

c1 = ohe_data.loc[ohe_data['SCAN_IMPLEMENTATION_IsNullScan'] == 1]
c3 = ohe_data.loc[ohe_data['SCAN_IMPLEMENTATION_ColumnVsColumn'] == 1]
c4 = ohe_data.loc[ohe_data['SCAN_IMPLEMENTATION_ColumnVsValue'] == 1]

fig, axs = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(15,5))
#ax=fig.add_axes([0,0,1,1])
fig.suptitle('ScanType')
axs[0].scatter(c1['SELECTIVITY'], c1['RUNTIME_NS'], color='r', alpha=0.5)
axs[0].set_title('IsNullScan')
axs[1].scatter(c3['SELECTIVITY'], c3['RUNTIME_NS'], color='g', alpha=0.5)
axs[1].set_title('ColumnVsColumn')
axs[2].scatter(c4['SELECTIVITY'], c4['RUNTIME_NS'], color='purple',alpha=0.5)
axs[2].set_title('ColumnVsValue')
#axs[0,0].set_ylim([0, 100000])

# for labels
fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axis
plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
plt.xlabel("Selectivity")
plt.ylabel("Time")
    
plt.show()


In [ ]:
# Now let's have a look at the TPCH data

In [ ]:
#Load all data
table_scan_data = pd.read_csv("../data/test/TableScan.csv")
table_data = pd.read_csv("../data/test/table_meta.csv")
columns_data = pd.read_csv("../data/test/column_meta.csv")
chunk_meta = pd.read_csv("../data/test/segment_meta.csv")

joined_data = table_scan_data.merge(table_data, on=["TABLE_NAME"], how="left")
joined_data = joined_data.merge(columns_data, on=["TABLE_NAME", "COLUMN_NAME"], how="left")

# only look at the encoding of the first chunk
chunkrows = chunk_meta.loc[(chunk_meta['CHUNK_ID'] == 0)]
joined_data = joined_data.merge(chunkrows, on=["TABLE_NAME", "COLUMN_NAME"], how="left")

joined_data = joined_data.rename(columns={"INPUT_ROWS_LEFT": "INPUT_ROWS", "CHUNK_SIZE": "MAX_CHUNK_SIZE",
                                          "COLUMN_DATA_TYPE": "DATA_TYPE", "ENCODING_TYPE": "ENCODING"})

# remove the rows with ExpressionEvaluator queries from the test data since we don't have any in the test data
joined_data = joined_data.loc[(joined_data['SCAN_IMPLEMENTATION'] != 'ExpressionEvaluator')]

# explicitly add selectivity
joined_data['SELECTIVITY'] = (joined_data['OUTPUT_ROWS'] / joined_data['INPUT_ROWS'])
joined_data['SELECTIVITY'].fillna(0, inplace=True)

In [ ]:
joined_data.describe()

In [ ]:
ohe_data = joined_data.drop(labels=['TABLE_NAME', 'COLUMN_NAME'], axis=1)
ohe_data = pd.get_dummies(ohe_data, columns=['SCAN_TYPE', 'DATA_TYPE', 'ENCODING', 'SCAN_IMPLEMENTATION', 'COMPRESSION_TYPE'])
ohe_data.head()

In [ ]:
plt.scatter(ohe_data['INPUT_ROWS'],ohe_data['RUNTIME_NS'])

In [ ]:
# looking at impact of the datatype

floats = ohe_data.loc[ohe_data['DATA_TYPE_float'] == 1]
ints = ohe_data.loc[ohe_data['DATA_TYPE_int'] == 1]
strings = ohe_data.loc[ohe_data['DATA_TYPE_string'] == 1]

fig=plt.figure()
ax=fig.add_axes([0,0,1,1])
ax.scatter(floats['INPUT_ROWS'], floats['RUNTIME_NS'], color='r', alpha=0.5, label='Float')
ax.scatter(ints['INPUT_ROWS'], ints['RUNTIME_NS'], color='b', alpha=0.5, label='Int')
ax.scatter(strings['INPUT_ROWS'], strings['RUNTIME_NS'], color='g', alpha=0.5, label='String')

plt.xlabel("#Input Rows")
plt.ylabel("Time [ns]")
plt.title('Datatype')
plt.legend(loc='upper center')
plt.show()

In [ ]:
# looking at impact of the compression

c1 = ohe_data.loc[ohe_data['COMPRESSION_TYPE_FixedSize1ByteAligned'] == 1]
c2 = ohe_data.loc[ohe_data['COMPRESSION_TYPE_FixedSize2ByteAligned'] == 1]

fig=plt.figure()
ax=fig.add_axes([0,0,1,1])
ax.scatter(c1['INPUT_ROWS'], c1['RUNTIME_NS'], color='r', label='FixedSize1ByteAligned')
ax.scatter(c2['INPUT_ROWS'], c2['RUNTIME_NS'], color='b', label='FixedSize2ByteAligned')
ax.set_xlabel('#Input Rows')
ax.legend(loc='upper center')
ax.set_ylabel('Time [ns]')
plt.title('Compression Type')
plt.show()

In [ ]:
# looking at impact of the compression
c1 = ohe_data.loc[ohe_data['SCAN_IMPLEMENTATION_ColumnBetween'] == 1]
c2 = ohe_data.loc[ohe_data['SCAN_IMPLEMENTATION_ColumnLike'] == 1]
c3 = ohe_data.loc[ohe_data['SCAN_IMPLEMENTATION_ColumnVsColumn'] == 1]
c4 = ohe_data.loc[ohe_data['SCAN_IMPLEMENTATION_ColumnVsValue'] == 1]


fig=plt.figure()
ax=fig.add_axes([0,0,1,1])
ax.scatter(c1['INPUT_ROWS'], c1['RUNTIME_NS'], color='r', label='SCAN_IMPLEMENTATION_ColumnBetween')
ax.scatter(c2['INPUT_ROWS'], c2['RUNTIME_NS'], color='b', label='SCAN_IMPLEMENTATION_ColumnLike')
ax.scatter(c3['INPUT_ROWS'], c3['RUNTIME_NS'], color='g', label='SCAN_IMPLEMENTATION_ColumnVsColumn')
ax.scatter(c4['INPUT_ROWS'], c4['RUNTIME_NS'], color='purple', label='SCAN_IMPLEMENTATION_ColumnVsValue')
ax.set_xlabel('#Input Rows')
ax.legend(loc='upper center')
ax.set_ylabel('Time [ns]')
plt.title('Compression Type')
plt.show()